In [3]:
import cv2
import numpy as np
from ultralytics import YOLO
import matplotlib as plt

In [5]:
lk_params=dict(winSize=(15,15),maxLevel=2,criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,0.03))

people_counter=0
line_y=250
font=cv2.FONT_HERSHEY_SIMPLEX


In [7]:
cap= cv2.VideoCapture('v1.mp4')
ret,frame=cap.read()
prev_gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

feature_params=dict(maxCorners=100,qualityLevel=0.3,minDistance=7,blockSize=7)
prevPts=cv2.goodFeaturesToTrack(prev_gray,mask=None,**feature_params)

mask=np.zeros_like(frame)

while True:
    ret, frame = cap.read()
    if not ret or frame is None:
        print("The frame could not get!")
        break  

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    newPts, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prevPts, None, **lk_params)

    if newPts is not None:
        good_new = newPts[status == 1]
        good_old = prevPts[status == 1]

        for new, old in zip(good_new, good_old):
            x_new, y_new = new.ravel()
            x_old, y_old = old.ravel()

            if y_old < line_y and y_new >= line_y:
                people_counter += 1

            mask = cv2.line(mask, (int(x_new), int(y_new)), (int(x_old), int(y_old)), (0, 255, 0), 2)
            frame = cv2.circle(frame, (int(x_new), int(y_new)), 5, (0, 0, 255), -1)

        frame = cv2.add(frame, mask)

 
    cv2.line(frame, (0, line_y), (frame.shape[1], line_y), (255, 0, 0), 2)
    cv2.putText(frame, f'Count: {people_counter}', (10, 50), font, 1, (0, 255, 255), 2)

    cv2.imshow("Pedestrian Tracker", frame)

    key = cv2.waitKey(30) & 0xFF
    if key == 27:
        break

    prev_gray = gray.copy()
    prevPts = good_new.reshape(-1, 1, 2)

cap.release()
cv2.destroyAllWindows()

 The frame could not get!
